In [36]:
import numpy as np
import cv2
import sys
import os
import math
np.set_printoptions(threshold=sys.maxsize)
import pickle
from skimage.transform import resize
from skimage import feature
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import glob
import imutils

In [10]:

""" items = os.listdir(folder_path)
folders = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]
for folder in folders:
    sign_path = os.path.join(folder_path,folder)
    sign_items = os.listdir(sign_path)
    # Filter out only the files (not directories)
    img_dir = [item for item in sign_items if os.path.isfile(os.path.join(sign_path, item))]
    if(folder == "cross_way" or folder == "parking" or folder == "roundabout" or folder == "traight"):
        for img_path in img_dir:
            img_p = os.path.join(sign_path,img_path)
            img = cv2.imread(img_p)
            image_lst.append(img)
            label_lst.append(folder) """


' items = os.listdir(folder_path)\nfolders = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]\nfor folder in folders:\n    sign_path = os.path.join(folder_path,folder)\n    sign_items = os.listdir(sign_path)\n    # Filter out only the files (not directories)\n    img_dir = [item for item in sign_items if os.path.isfile(os.path.join(sign_path, item))]\n    if(folder == "cross_way" or folder == "parking" or folder == "roundabout" or folder == "traight"):\n        for img_path in img_dir:\n            img_p = os.path.join(sign_path,img_path)\n            img = cv2.imread(img_p)\n            image_lst.append(img)\n            label_lst.append(folder) '

In [37]:
folder_path = "Sign"
def getData(folder):
    img_lst = []
    lb_lst = []
    sign_path = os.path.join(folder_path,folder)
    sign_items = os.listdir(sign_path)
    # Filter out only the files (not directories)
    img_dir = [item for item in sign_items if os.path.isfile(os.path.join(sign_path, item))]
    for img_path in img_dir:
        img_p = os.path.join(sign_path,img_path)
        img = cv2.imread(img_p)
        img_lst.append(img)
        lb_lst.append(folder)
    return img_lst,lb_lst

In [38]:
forbit,lbfb = getData("forbit")
stop,lbs = getData("stop")



In [41]:
image_lst = forbit[24:] + stop
label_lst = lbfb[24:] + lbs

In [42]:
print(len(label_lst))

115


In [43]:
#for red
def preprocess_img(img):
    # Convert to grayscale if the image is in color
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = img_gray.astype(np.float32)
    resized_img = resize(
      img_gray,
      output_shape=(64,64),
      anti_aliasing=True
  )
    # Extract HOG features
    hog_feature = feature.hog(resized_img, orientations=4, pixels_per_cell=(4, 4),
                cells_per_block=(2, 2), transform_sqrt = False, block_norm="L1", visualize=False, feature_vector=True)


    return hog_feature

In [11]:
# for blue
def preprocess_img(img):
    # Convert to grayscale if the image is in color
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = img_gray.astype(np.float32)
    resized_img = resize(
      img_gray,
      output_shape=(64,64),
      anti_aliasing=True
  )
    # Extract HOG features
    hog_feature = feature.hog(resized_img, orientations=4, pixels_per_cell=(4, 4),
                            cells_per_block=(2, 2), transform_sqrt = False, block_norm="L1", visualize=False, feature_vector=True)

    return hog_feature

In [44]:
img_feature_lst = []
for img in image_lst:
  hog_feature = preprocess_img(img)
  img_feature_lst.append(hog_feature)
img_features = np.array(img_feature_lst)
print("X shape: ")
print(img_features.shape)

X shape: 
(115, 3600)


In [45]:
classifier = SVC(gamma='scale', decision_function_shape= 'ovo',probability = True)

In [46]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label_lst)


In [47]:
print(encoded_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]


In [48]:
random_state = 0
test_size = 0.2
is_shuffle = True
X_train, X_val, y_train, y_val = train_test_split(
    img_features,encoded_labels,
    test_size = test_size,
    random_state = random_state,
    shuffle = is_shuffle
)

In [49]:
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_val)
score = accuracy_score(y_pred, y_val)
print("Accurancy: ",score)

Accurancy:  1.0


In [53]:
with open("red.pkl", "wb") as f:
    pickle.dump(classifier, f)


In [54]:
lst = []
for input_img in stop:
    normalized_img = preprocess_img(input_img)
    y_pred = classifier.predict([normalized_img])[0]
    lst.append(y_pred)
print(len(lst))
print(lst.count(1))

50
0


In [33]:
with open("red.pkl", "rb") as f:
    red_clf = pickle.load(f)

In [55]:
input_img = cv2.imread("image_18.jpg")
normalized_img = preprocess_img(input_img)
y_pred = classifier.predict([normalized_img])
print(f'Normal prediction: {y_pred}')
y_pred_prob = classifier.predict_proba([normalized_img])
print(max(y_pred_prob[0]))
prediction = np.argmax(y_pred_prob)
y_pred_prob = [f'{p:.10f}' for p in y_pred_prob[0]]
print(f'Probability: {y_pred_prob}')
print(f'Class with highest: {label_encoder.classes_[prediction]}') 

Normal prediction: [1]
0.9771326371811536
Probability: ['0.0228673628', '0.9771326372']
Class with highest: stop
